In [15]:
import numpy as np
import cvxpy as cp

In [42]:
def estimate_params(V, U):
    if len(V) != len(U):
        raise ValueError("Arrays must have the same size")
    T = len(V)
    #gammas = np.ones((2, 2))
    '''
    iters = 2000
    for i in range(iters):
        grad = np.zeros((2, 2))
        for d in range(T - 1):
            grad[0, 0] += gammas[0, 0] * np.square(V[d]) - V[d] * V[d+1]
            grad[0, 1] += gammas[0, 1] * np.square(U[d]) - U[d] * V[d+1]
            
            grad[1, 0] += gammas[1, 0] * np.square(V[d]) - V[d] * U[d+1]
            grad[1, 1] += gammas[1, 1] * np.square(U[d]) - U[d] * U[d+1]
        stepsize = 0.5 / (i+1)
        gammas = gammas - grad * stepsize
    '''
    gammas = cp.Variable((2, 2))
    constraints = [gammas >= 0.1]
    expression = 0
    for d in range(T-1):
        expression += cp.sum_squares(np.array([V[d+1], U[d+1]]) - gammas @ np.array([V[d], U[d]]))
    problem = cp.Problem(cp.Minimize(expression), constraints)
    problem.solve()
    
    opt_val = gammas.value
    
    return opt_val

In [28]:
true_gammas = np.array([[0.9, 1.05], [0.8, 1.1]])
true_gammas

array([[0.9 , 1.05],
       [0.8 , 1.1 ]])

In [40]:
T = 25
U = np.zeros(T)
V = np.zeros(T)
V[0] = 2
U[0] = 5
for i in range(T-1):
    noise1 = np.random.normal(0, np.sqrt(V[i]))
    noise2 = np.random.normal(0, np.sqrt(U[i]))
    V[i+1] = V[i] * true_gammas[0, 0] + U[i] * true_gammas[0, 1] + noise1
    U[i+1] = V[i] * true_gammas[1, 0] + U[i] * true_gammas[1, 1] + noise2


In [41]:
gammaest = estimate_params(V, U)
gammaest

SolverError: Solver 'OSQP' failed. Try another solver, or solve with verbose=True for more information.